This notebook is just that - a list of notes. It can be used ot reconstruct all lists and tables from scratch, but in practice I don't want to do that every time. Instead, I can load the intermediate data tables for most purposes.

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table, QTable
from astropy import table
from astropy.coordinates import SkyCoord, Distance, Latitude, Longitude, Distance
from astropy.time import Time

from astroquery.simbad import Simbad

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
names = Table.read('targetnames.txt', format='ascii.no_header', delimiter='|', names=['target'])

In [ ]:
TargetSimbad = Simbad()
TargetSimbad.add_votable_fields('propermotions', 'distance', 'parallax', 'flux(B)', 'flux(V)', 'rv_value', 'sp')
TargetSimbad.add_votable_fields('ra(2;A;ICRS;J2000;2000)', 'dec(2;D;ICRS;J2000;2000)')

In [ ]:
targets_download = TargetSimbad.query_objects(names['target'])

In [ ]:
targets = table.hstack([names, targets_download], join_type='exact')
# translate units to astropy
for col in targets.colnames:
    if str(targets[col].unit) == '"h:m:s"':
        targets[col] = Longitude(targets[col], unit=u.hourangle)
    elif str(targets[col].unit) == '"d:m:s"':
        targets[col] = Latitude(targets[col], unit=u.deg)
# For Distance_distance, units are in separate column 
        
targets = QTable(targets)

In [ ]:
targets['Distance_unit'][targets['Distance_unit'] == ''] = 'pc'
targets['distance'] = u.Quantity([row['Distance_distance'] * u.Unit(row['Distance_unit']) for row in targets])
# Some distances suddenly become 0, where the input was masked. We deal with those next.


# Distances are typically taken from GAIA, but some bright stars are missing. 
# For those, we calculate the distance from the parallax (from HIPPARCOS)
ind = targets['Distance_distance'].mask
ind = targets['Distance_perr'].mask
targets['distance'][ind] = targets['PLX_VALUE'].to(u.pc, equivalencies=u.parallax())[ind]
targets['Distance_method'][ind] = 'paral'
targets['Distance_perr'][ind] = ((targets['PLX_VALUE'] - targets['PLX_ERROR']).to(u.pc, equivalencies=u.parallax()) 
                                 - targets['distance'])[ind]
targets['Distance_merr'][ind] = ((targets['PLX_VALUE'] + targets['PLX_ERROR']).to(u.pc, equivalencies=u.parallax()) 
                                 - targets['distance'])[ind]
targets['Distance_bibcode'][ind] = targets['PLX_BIBCODE'][ind]
# finally: Clean up non-quantity input columns
targets.remove_columns(['Distance_distance', 'Distance_unit'])

In [ ]:
# Convert byte objects into strings, because we can't save byte objetcs to fits later
for c in targets.colnames:
    if targets[c].dtype == np.object:
        targets[c] = [s.decode() for s in targets[c]]

In [ ]:
targets['coord'] = SkyCoord(ra=targets['RA_2_A_ICRS_J2000_2000'],
                            dec=targets['DEC_2_D_ICRS_J2000_2000'],
                            distance=targets['distance'],
                            pm_ra_cosdec=targets['PMRA'],
                            pm_dec=targets['PMDEC'],
                            frame='icrs', obstime='J2000', equinox='J2000')

In [ ]:
targets.write('targets.csv', format='ascii.ecsv')

In [ ]:
# Note: I doubt that the proper motion in the table is serialized correctly, but can always reconstruct
# from the other columns.

## Other source properties

search.py in this directory pulls in other properties fomr e.g. GAIA and Vizier for mass, bolometric luminosity etc. Eventually, I need that, but first I continue with the X-ray analysis and leave this chapter empty.

## List of datasets and properties

In [ ]:
from glob import glob
import os
import re

from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
def header_to_table(filename, ext=0):
    header = fits.getheader(filename, ext=ext)
    h = {k: [header[k]] for k in header.keys()}
    h['filename'] = [filename]
    h['obsdirname'] = [os.path.basename(filename)]
    # The following can have different shapes, which prevents merging
    for key in ['HISTORY', 'COMMENT']:
         if key in h:
            del h[key]
    return Table(h)

In [ ]:
tablist = []
for f in glob('data/Chandra/*/repro/*evt2*'):
    tablist.append(header_to_table(f, ext=1))
chandra_data = table.vstack(tablist)  

In [ ]:
chandra_data['pnt_coords'] = SkyCoord(ra=chandra_data['RA_PNT'], 
                                      dec=chandra_data['DEC_PNT'],
                                      unit=(u.deg, u.deg))

idx, d2d, d3d = chandra_data['pnt_coords'].match_to_catalog_sky(targets['coord'])

chandra_data['target'] = targets['target'][idx]
chandra_data['target_coord'] = targets['coord'][idx]
chandra_data['distance_from_pnt'] = d2d

In [ ]:
from ciao_contrib.region.check_fov import FOVFiles

chandra_data['FOV'] = False
for row in chandra_data: 
    my_obs = FOVFiles(row['filename'].replace('evt2', 'fov1'))
    ii = my_obs.inside(row['target_coord'].ra.deg, row['target_coord'].dec.deg)
    if len(ii) > 0:
        row['FOV'] = True
        
# Only keep those sources that have our target in FOV.
# Need to check for XMM, too, once I include that.
chandra_data = chandra_data[chandra_data['FOV']]

In [ ]:
chandra_data['target', 'distance_from_pnt', 'FOV', 'TELESCOP', 'OBS_ID', 'INSTRUME', 'GRATING', 'EXPOSURE']

In [ ]:
def find_image_in_slew_match(obsid, targets):
    '''Step throu all band 8 slew exposure maps.
    For each map, check if one of the target objects i in the exposure map,
    and if so, if the exposure time is > 0 (i.e. it truely is observed).
    If an object is part of more than one image, select he image where ii
    is further away form the edge.
    
    Returns
    -------
    expmapout : list of strings
        filename of a expmap file that contains a target
    '''
    d = []
    expmaps = []
    objects = []
    for expmap in glob('data/XMM/' + obsid + '/*EXPMAP8???.ds'): # band 8 is all energies
        with fits.open(expmap) as hdus:
            wcs = WCS(hdus[0].header)
            dat = hdus[0].data
        y, x = wcs.all_world2pix(targets['coord'].ra, targets['coord'].dec, 0, ra_dec_order=True)
        object_in_image = (x > 0) & (x < dat.shape[0]) & (y > 0) & (y < dat.shape[1])
        for ind in object_in_image.nonzero()[0]:
            # Check exposure time is > 0, i.e. object is in slew path
            if dat[int(x[ind]), int(y[ind])] > 0:
                # Get distance from image center
                d.append((x[ind] - dat.shape[0]/2)**2 + (y[ind] - dat.shape[1]/2)**2)
                expmaps.append(expmap)
                objects.append(targets['target'][ind])
    # Now select the best images if an object is in more than one 
    # (image overlap at the edges, but this is really the same data)
    expmapout = []
    # Some comparisons are eaiser ot write if it's all turned into numpy arrays
    objects = np.array(objects)
    d = np.array(d)
    expmaps = np.array(expmaps)
    for o in set(objects):
        oind = objects == o
        minind = np.argmin(d[oind])
        expmapout.append(expmaps[oind][minind])
    return expmapout

In [ ]:
tablist = []

for d in glob('data/XMM/*'):
    obsid = os.path.basename(d)
    if re.match('^[0-8]', obsid):
        infiles = glob(d + '/images/*')
        if len(infiles) == 0:
            infiles = glob(d + '/rgs/pi*fit')
        if len(infiles) > 0:
            tablist.append(header_to_table(infiles[0]))
        else:
            # I downloaded these ObsIDS, but they seem not to contain any data.
            # I also can't find them in a "by obsid" search in the XSA.
            # Might be aborted or cal with unusual setups?
            print('No data found:', obsid)
    elif obsid.startswith('9'):
        # slew
        expfiles = find_image_in_slew_match(obsid, targets)
        if len(expfiles) == 0 :
            print('Slew with no target in path:', obsid)
        for e in expfiles:
            tablist.append(header_to_table(e))
    else:
        # Not XMM data, e.g. filename of a script that's left over in directory
        pass

In [ ]:
xmm_data = table.vstack(tablist)

In [ ]:
# Need to remove mixin coordinate column before stacking for astropy < 4.2
chandra_data.remove_columns(['pnt_coords', 'target_coord'])
all_data = table.vstack([chandra_data, xmm_data])
# Select columns to keep. This includes Chandr and XMM specific keywords, but most keywords
# are OGIP standard and appear in both missions.
all_data.keep_columns(['TELESCOP', 'OBS_ID', 'INSTRUME', 'GRATING', 'EXPOSURE',
    'TITLE', 'DS_IDENT', 'DETNAM', 'EXP_ID', 'EXPIDSTR',
    'FILTER', 'DATAMODE', 'DATE-OBS', 'DATE-END', 'OBS_MODE', 'OBSERVER', 'OBJECT', 
    'RA_OBJ', 'DEC_OBJ', 'RA_NOM', 'DEC_NOM', 'RA_PNT', 'DEC_PNT', 'PA_PNT', 'ROLL_PNT',
    'RA_NOM', 'DEC_NOM', 'ROLL_NOM', 'EXPOSURE',
                       'SIM_X', 'SIM_Y', 'SIM_Z', 'DY_AVG', 'DZ_AVG', 'DTH_AVG',
    'filename', 'obsdirname'])

In [ ]:
all_data['pnt_coords'] = SkyCoord(ra=all_data['RA_PNT'], 
                                      dec=all_data['DEC_PNT'],
                                      unit=(u.deg, u.deg))

idx, d2d, d3d = all_data['pnt_coords'].match_to_catalog_sky(targets['coord'])

all_data['target'] = targets['target'][idx]
all_data['target_coord'] = targets['coord'][idx]
all_data['distance_from_pnt'] = d2d
# There are a few XMM calibration observations with mismatches between requested
# and true pointing that happend to be too far away to be useful here.
all_data = all_data[all_data['distance_from_pnt'] < 1 * u.deg]

In [ ]:
all_data

## Cutouts and preview checks

In [ ]:
import ciao_contrib.runtool as rt
from coords.chandra import cel_to_chandra
from astropy.time import Time

row = all_data[all_data['TELESCOP'] == 'CHANDRA'][0]
# Many of the objects have high proper motion
target_at_obs = row['target_coord'].apply_space_motion(new_obstime=Time(row['DATE-OBS']))

In [ ]:
chan_coords = cel_to_chandra({k: row[k] for k in row.colnames}, target_at_obs.ra.deg, target_at_obs.dec.deg)

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
hdus = fits.open("data/XMM/0843151001/mos/3642_0843151001_EMOS1_S001_ImagingEvts.ds")

In [ ]:
wcs = WCS(hdus[1].header, keysel=['pixel'], naxis=['celestial'])

In [ ]:
wcs

In [ ]:
evts = Table(hdus[1].data)

In [ ]:
h, x_edges, y_edges = np.histogram2d(evts['X'], evts['Y'], bins=400)

In [ ]:
from regions import CircleSkyRegion, CirclePixelRegion
from astropy.coordinates import Angle, SkyCoord

In [ ]:
region = CircleSkyRegion(SkyCoord('19:30', '51:40', unit=(u.hourangle, u.deg)), Angle(.1 * u.deg))

In [ ]:
artist = region.to_pixel(wcs).as_artist(edgecolor='r')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=wcs, aspect=1)
ax.imshow(h.T, extent=[x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]])
ax.add_artist(artist)

In [ ]:
evts